In [ ]:
# Load required packages
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc

In [ ]:
# Create connection to AdventureWorksDW
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=localhost\SQLEXPRESS;'
                        'Database=AdventureWorksDW2019;'
                        'Trusted_Connection=yes;')

In [ ]:
# Define sql to query SalesView
sql = "SELECT * FROM SalesView"

# Load SalesView
df = pd.read_sql(sql, conn)
df.head()

In [ ]:
# Clean column names
df.rename(
    columns = {
        "OrderYear": "year",
        "OrderMonth": "month",
        "SalesAmount": "sales", 
        "ProductKey": "item", 
        "SalesTerritoryKey": "sales_terr_key"
    }
    , inplace = True 
)

In [ ]:
# Add day and day of week to df
df["day"] = pd.to_datetime(df["OrderDate"]).dt.day
df["day_of_week"] = pd.to_datetime(df["OrderDate"]).dt.day_of_week
df.head()

In [ ]:
# Group sales by day by item by store
sales_df = df.groupby(["sales_terr_key", "item", "OrderDate", "year", "month", "day", "day_of_week"], as_index = False)["sales"].sum()
sales_df.sort_values(["sales_terr_key", "item", "year", "month", "day", "day_of_week"]).head()

In [ ]:
# Filter for 2013 data
sales_df = sales_df[(sales_df["OrderDate"] <= '2013-12-31') & (sales_df["OrderDate"] >= '2013-01-01')]

# Convert OrderDate to datetime
sales_df["OrderDate"] = pd.to_datetime(sales_df["OrderDate"])

In [ ]:
# Get unique list of territories & items
unique_val_df = sales_df[["sales_terr_key", "item"]].drop_duplicates()

# Add join key
unique_val_df['key'] = 0
unique_val_df.head()

In [ ]:
# Create full date range df
date_df = pd.DataFrame(pd.date_range(min(sales_df["OrderDate"]), pd.to_datetime('2014-01-15')), columns = ["OrderDate"])

# Add join key
date_df['key'] = 0

In [ ]:
# Merge items w/ dates
full_date_df = date_df.merge(unique_val_df, how = "outer", on = 'key').sort_values(["sales_terr_key", "item", "OrderDate"])
full_date_df.head()

In [ ]:
# Create full date range sales_df
final_sales_df = full_date_df.merge(sales_df, how = "left", on = ['sales_terr_key', 'item', 'OrderDate'])

# Clean sales NA values
final_sales_df["sales"] = final_sales_df["sales"].fillna(0)

# Keep required columns
final_sales_df = final_sales_df[["OrderDate", "sales_terr_key", "item", "sales"]]
final_sales_df.tail()

In [ ]:
# Calculate rolling average
final_sales_df["rolling_sales_pred"] = final_sales_df.groupby(['sales_terr_key', 'item'])['sales'].transform(lambda x: x.rolling(15, 1).mean())

In [ ]:
# Calculate 15 days ago
min_date = max(final_sales_df["OrderDate"]) - datetime.timedelta(15)

# Filter final_sales_df for last 15 days
results_df = final_sales_df[final_sales_df["OrderDate"] > min_date]

# Change OrderDate type
results_df["OrderDate"] = results_df["OrderDate"].dt.date

# Select final columns
results_df = results_df[["OrderDate", "sales_terr_key", "item", "rolling_sales_pred"]]
results_df.head()

In [ ]:
# Create python cursor
cursor = conn.cursor()

# Drop results table
cursor.execute("DROP TABLE ForecastTable")

# Create results table if needed
cursor.execute(
    
""" 
CREATE TABLE ForecastTable(
    OrderDate            VARCHAR(50) NOT NULL
    ,sales_terr_key      VARCHAR(4) NOT NULL
    ,item                VARCHAR(30) NOT NULL
    ,rolling_sales_pred  NUMERIC(10,5) NOT NULL
    )
    
"""
)

In [ ]:
# Insert Dataframe into SQL Server:
for i in range(len(results_df)):
    
    cursor.execute("INSERT INTO ForecastTable (OrderDate, sales_terr_key, item, rolling_sales_pred) values(?,?,?,?)"
    , str(results_df.iloc[i][0])
    , str(results_df.iloc[i][1])
    , str(results_df.iloc[i][2])
    , float(results_df.iloc[i][3])
    )
conn.commit()

In [ ]:
# Close database connection
cursor.close()
print('Insert Complete')